In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import tmdbsimple as tmdb
import os
import re
import sys
import pickle
import datetime
import time
import pdbs
import uuid
from tqdm.autonotebook import tqdm
tqdm.pandas()
import sqlite3
from fuzzywuzzy import fuzz
%load_ext autoreload
%autoreload 2
sys.path.append(os.path.abspath('../src'))

/home/owner/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  del sys.path[0]
/home/owner/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/owner/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import gather_data as data_agg

In [11]:
# netflix_path = '../Data/NetflixViewingHistory.csv' ; user_id = 'malcolm' # MT Small pt1
# netflix_path = '/home/owner/Downloads/NetflixViewingHistory (1).csv'; user_id='vinny' # Lauryn, Vinny (big)
# netflix_path = '/home/owner/Downloads/NetflixViewingHistory (2).csv'; user_id='malcolm2' # MT Small pt2

with open('../Data/api_key.pkl', 'rb') as hnd:
    key = pickle.load(hnd)['api_key']
db_location = '../Data/netflix_viewing1.db'

In [12]:
def connect_db():
    conn = sqlite3.connect(db_location)
    return(conn)
search= tmdb.Search()

In [13]:
conn = connect_db()

## Build Test set 

In [15]:
netflix_path = '../Data/test.csv'
user_id='test'

## Demo Run Through

In [16]:
data_pipeline = data_agg.DataPipeline(netflix_path, key, db_location, id=user_id)
# data_pipeline = data_agg.DataPipeline(test_path, key, db_location)
data_pipeline.execute()
# netflix_df = data_pipeline.parse_netflix(netflix_path)
# # netflix_df = data_pipeline.parse_netflix(test_path)

# netflix_df = data_pipeline.get_netflix_df(conn, netflix_path, )
# movie_df   = data_pipeline.generate_movie_df(netflix_df)
# tv_df      = data_pipeline.generate_tv_df(netflix_df)


User: test
Total number of TV Show + Movies: 2
Total number of movies: 1
Found 1 matches in db 
Did not find 0 titles, querying TMDB for results
Total number of Episodes of TV Shows: 1
Number of unique TV Shows: 1
Found 1 matches in db 
Did not find 0 titles, querying TMDB for results


In [ ]:
movie_df

In [ ]:
tv_df.columns

In [ ]:
[x for x in tv_df.columns if 'genre' in x.lower()]

## Cleaning

In [ ]:
text1 = "The Jungle Book (2016)"

In [ ]:
re.sub(".*?\((.*?)\)", '',text1)

In [ ]:
regex = re.compile(".*?\((.*?)\)")
result = re.findall(regex, text1)
result

In [ ]:
re.sub("[\(\[].*?[\)\]]", "", text1)


In [ ]:
data_pipeline.all_movies_results_df

In [ ]:
movie_df.head()

In [ ]:
raw_movies = netflix_df[netflix_df['TV Show flag'] == 'Movie']
raw_movies

## Practice using API 

In [ ]:
search_term = "Gladiator"
search_results = search.movie(query=search_term)
search_results

In [ ]:
names = [{'New Title': x['title']
       , 'TMDB id' : x['id']
       , 'Popularity':x['popularity']
       , 'Cosine Distance': fuzz.ratio(search_term, x['title'])} for x in search_results['results']]
names_df = pd.DataFrame(names).sort_values(['Cosine Distance', 'Popularity'], ascending=False)
keeper_id = names_df.iloc[0]['TMDB id']
keeper_id

In [ ]:
pd.DataFrame(names).sort_values(['Cosine Distance', 'Popularity'], ascending=False)

In [ ]:
full_movie_results = tmdb.Movies(keeper_id).info()
full_movie_results

In [ ]:
full_movie_results

In [ ]:
normal_movie_fields = ['budget', 'homepage', 'imdb_id', 'overview', 'popularity' \
            , 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'tagline']
movie_results = {attr: full_movie_results.get(attr) for attr in normal_movie_fields}
movie_results

In [ ]:
search_results['results'][4]

## Redo apply and practice with DB 

In [ ]:
raw_movies.head()

In [ ]:
def _get_best_tmdb_search_result(search_term, search_results):
    """
    Find best TMDB match based on name and popularity
    :return:
    """
    names = [{'New Title': x.get('title', x.get('name'))
                 , 'TMDB id': x['id']
                 , 'Popularity': x['popularity']
                 , 'Cosine Distance': fuzz.ratio(search_term, x.get('title', x.get('name')))} for x in search_results['results']]
    names_df = pd.DataFrame(names).sort_values(['Cosine Distance', 'Popularity'], ascending=False)
    keeper_id = names_df.iloc[0]['TMDB id']
    return(keeper_id)


def _get_movie_API_results2(movie_title):
    """
    Query TMDB API for movie title, find best result, return selected fields
    :param movie_title:
    :return:
    """
    # Select requested fields from response
    normal_movie_fields = ['original_title' ,'budget', 'homepage', 'imdb_id', 'overview', 'popularity' \
        , 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'tagline']

    # Find the Movie in TMDB
    search_results = search.movie(query=movie_title)
    n_results = len(search_results['results'])
    #     print("N Results: ", n_results)

    # Handle 0, 1, multiple tmdb search results
    if n_results == 0:
        movie_results = {key: np.nan for key in normal_movie_fields}
        movie_results['Number of Search Results'] = n_results
        movie_results['Input Movie Title'] = movie_title
        movie_results['Cosine Distance'] = np.nan
        movie_results['Date Pulled'] = datetime.datetime.now().date()
        movie_results = pd.Series(movie_results)

        return (movie_results)

    elif n_results == 1:
        temp_id = search_results['results'][0]['id']
    elif n_results > 1:
        temp_id =  _get_best_tmdb_search_result(movie_title, search_results)

    # Get TMDB attributes
    full_movie_results = tmdb.Movies(temp_id).info()
    movie_results = {attr: full_movie_results.get(attr) for attr in normal_movie_fields}

    # Append number of search results (incase there are multiple and we choose the wrong one)
    movie_results['Number of Search Results'] = n_results
    movie_results['Input Movie Title'] = movie_title
    movie_results['Cosine Distance'] = fuzz.ratio(movie_title, movie_results.get('original_title'))
    movie_results['Date Pulled'] = datetime.datetime.now().date()

    
    # Prepare to export
    movie_results = pd.Series(movie_results)
    new_column_order = normal_movie_fields.copy()
    new_column_order.insert(0, 'Input Movie Title')
    additional_cols = ['Number of Search Results', 'Cosine Distance', 'Date Pulled']
    new_column_order.extend(additional_cols)
    movie_results = movie_results[new_column_order]

    return (movie_results)


In [ ]:
df2 = raw_movies['Title'].apply(lambda x:_get_movie_API_results2( movie_title = x))
df2.head()

In [ ]:
df2.shape

In [ ]:
df2['original_title'].isna().sum()

In [ ]:
df2.iloc[:20].to_sql('Movies', conn, index=False, if_exists='replace')

In [ ]:
data_pipeline = data_agg.DataPipeline(netflix_path, key, db_location)
# data_pipeline.execute()
netflix_df = data_pipeline.parse_netflix(netflix_path)
movie_df   = data_pipeline.generate_movie_df(netflix_df)
# tv_df      = data_pipeline.generate_tv_df(netflix_df)


In [ ]:
if 1==1: print ("hello")

## Demo read from DB 

In [ ]:
df1 = pd.read_sql_query('select * from Movies', conn)
df1.shape

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
raw_movies

In [ ]:
df3 = raw_movies[['Title', 'Date Watched']].copy()

In [ ]:
df3.shape

In [ ]:
print("Unique input movies: ", df3['Title'].nunique(), ' Shape: ', df3.shape[0])
print("Unique database movies: ", df1['Input Movie Title'].nunique(), " Shape: ", df1.shape[0])

In [ ]:
missing_ids = pd.merge(df3, df1, left_on = 'Title', right_on='Input Movie Title', how='left')

In [ ]:
missing_ids.shape

## TV Show Version 

In [ ]:
tv_shows = netflix_df[netflix_df['TV Show flag'] == 'TV Show']

In [ ]:
tv_shows

In [ ]:
demo_tv_show = tv_shows['Show Name'].unique()[0]
tv_shows['Show Name'].unique()


In [ ]:
demo_tv_show

In [ ]:
tv_search_results = search.tv(query=demo_tv_show)
tv_search_results

In [ ]:
tv_show_id = tv_search_results['results'][0]['id']
tv_show_id

In [ ]:
tv_show_query = tmdb.TV(tv_show_id).info()
tv_show_query

In [ ]:
all_tv_episodes = netflix_df[netflix_df['TV Show flag'] == 'TV Show']
print(f"Total number of Episodes of TV Shows: {all_tv_episodes.shape[0]}")
user_tv_shows = all_tv_episodes.groupby('Show Name')

In [ ]:
all_tv_episodes

In [ ]:
def get_series_watched_gb(df):
    
    out = {}
    out['Number of Episodes'] = df.shape[0]
    out['First Episode Watcheed'] = df['Date Watched'].min()
    out['Last Episode Watched'] = df['Date Watched'].max()
    out['Number of Seasons Watched'] = df['Season'].nunique()
    
    # Season Metrics 
    season_gb = df.groupby('Season')
    season_episode_cnt = season_gb.size()
    time_to_watch_season = (season_gb['Date Watched'].max() - season_gb['Date Watched'].min())

    # Watch Rate is time/# of episodes
    episode_watch_rate = time_to_watch_season/season_gb['Date Watched'].size() 
    
    # Longest Time to Watch 
    out['Longest Time to Watch Season'] = time_to_watch_season.max()
    out['Season took Longest to Watch'] = time_to_watch_season.idxmax()
    out['# of Episode for Longest to Watch'] = season_episode_cnt.loc[time_to_watch_season.idxmax()]
    
    # Longest Days/Episode
    out['Longest Days/Episode'] = episode_watch_rate.max()
    out['Season with Longest Days/Episode'] = episode_watch_rate.idxmax()
    out['# of Episode for Longest Days/Episode'] = season_episode_cnt.loc[episode_watch_rate.idxmax()]

    # Shortest Time to Watch
    out['Shortest Time to Watch Season'] = time_to_watch_season.min()
    out['Season took Shortest to Watch'] = time_to_watch_season.idxmin()
    out['# of Episode for Shortest Time to Watch'] = season_episode_cnt.loc[time_to_watch_season.idxmin()]
    
    # Shortest Days/Episode
    out['Shortest Days/Episode'] = episode_watch_rate.min()
    out['Season with Shortest Days/Episode'] = episode_watch_rate.idxmin()
    out['# of Episode for Shortest Days/Episode'] = season_episode_cnt.loc[episode_watch_rate.idxmin()]

    
    out_series = pd.Series(out)
    return(out_series)

### Testing Season Function

In [ ]:
temp_df = all_tv_episodes[all_tv_episodes['Show Name'] == 'Black Mirror']
temp_df.head()

In [ ]:
temp_df.shape

In [ ]:
season_gb = temp_df.groupby('Season')

In [ ]:
((season_gb['Date Watched'].max() - season_gb['Date Watched'].min())/season_gb['Date Watched'].size())

In [ ]:
season_gb.size()

## See Apply Loop

In [ ]:
out_df = user_tv_shows.apply(get_series_watched_gb)
out_df

In [ ]:
out_df.shape

In [ ]:
out_df[out_df['Number of Seasons Watched'] > 2].shape

In [ ]:
good_series = out_df[(out_df['Number of Seasons Watched'] > 1) 
                     & (out_df['Shortest Days/Episode'] > datetime.timedelta(minutes=15))]
good_series

In [ ]:
good_series.shape

In [ ]:

def _get_TV_show_details(tv_show_name):

    # Select requested_fields from response
    normal_tv_show_fields = ['n_production_companies', 'primary_production_co', 'runtime', 'release_date'
        , 'n_network', 'primary_network', 'homepage', 'overview', 'popularity', 'vote_average', 'vote_count'
        , 'in_production', 'type', 'status', 'number_of_seasons', 'number_of_episodes']

    # Search TMDB by TV show name
    search_results = search.tv(query=tv_show_name)
    n_results = len(search_results['results'])

    # Parse search response
    if n_results == 0:
        tv_show_results = {key:np.nan for key in normal_tv_show_fields}
        tv_show_results['Number of Search Results'] = n_results
        tv_show_results['Input TV Show Title'] = tv_show_name
        tv_show_results['Cosine Distance'] = np.nan
        tv_show_results['Date Pulled'] = datetime.datetime.now().date()
        tv_show_results = pd.Series(tv_show_results)
        return(tv_show_results)

    elif n_results == 1:
        temp_id = search_results['results'][0]['id']
    elif n_results > 1:
        temp_id = _get_best_tmdb_search_result(tv_show_name, search_results)

    full_tv_show_results=tmdb.TV(temp_id).info()

    # Fill dictionary with response items directly
    normal_tv_fields = ['homepage', 'overview', 'popularity' \
        , 'vote_average', 'vote_count', 'name', 'id'
        , 'in_production', 'type', 'status', 'number_of_seasons', 'number_of_episodes']
    tv_show_results = {attr: full_tv_show_results.get(attr) for attr in normal_tv_fields}

    # Attributes that require some preprocessing
    tv_show_results['n_production_companies'] = len(full_tv_show_results['production_companies'])
    tv_show_results['primary_production_co'] = '' if len(full_tv_show_results['production_companies']) == 0 else \
    full_tv_show_results['production_companies'][0]['name']
    tv_show_results['runtime'] = 0 if len(full_tv_show_results) == 0 else full_tv_show_results['episode_run_time'][0]
    tv_show_results['release_date'] = full_tv_show_results['first_air_date']
    tv_show_results['n_network'] = len(full_tv_show_results['networks'])
    tv_show_results['primary_network'] = '' if len(full_tv_show_results['networks']) == 0 else \
    full_tv_show_results['networks'][0]['name']

    # Append additional fields
    tv_show_results['Number of Search Results'] = n_results
    tv_show_results['Input TV Show Title'] = tv_show_name
    tv_show_results['Cosine Distance'] = fuzz.ratio(tv_show_name, tv_show_results.get('original_name'))
    tv_show_results['Date Pulled'] = datetime.datetime.now().date()

    return (tv_show_results)


In [ ]:
tv_df = data_pipeline.generate_tv_df(netflix_df)


In [ ]:
df1 = data_pipeline.all_tv_shows_results_df
df1.head()

In [ ]:
df1.shape

In [ ]:
tv_show_details = data_pipeline._get_TV_show_details('Medici')
tv_show_details

In [ ]:
tv_show_details.index.tolist()

In [ ]:
tv_tbl = pd.read_sql('select * from TV_Shows', conn)
tv_tbl.shape

In [ ]:
tv_tbl.head()

In [ ]:
tv_tbl['homepage'].nunique()

In [ ]:
tv_tbl.columns

In [ ]:
deduped = tv_tbl.drop_duplicates()
deduped.shape

In [ ]:
deduped.to_sql('TV_Shows', conn, index=False, if_exists='replace')

## Pandas SQL 

In [ ]:
tbl1 = pd.DataFrame(['a', 'b', 'c', 'd'], index = [1,2,3,4])
tbl1

In [ ]:
tbl2 = pd.DataFrame(['c', 'd', 'e'], index=[1, 2, 3])
tbl2

In [ ]:
pd.merge(tbl1, tbl2, indicator='i', how='left')

In [ ]:
def get_db_tables(db_location):
    con = sqlite3.connect(db_location)
    cursor = con.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tbls = cursor.fetchall()
    tbls = [x[0] for x in tbls]
    return(tbls)


In [ ]:
tbl1.to_sql('temp', conn)

In [ ]:
get_db_tables(db_location)

In [ ]:
conn.cursor().execute('select * from TV_Shows').fetchall()

In [ ]:
sql2 = 'PRAGMA table_info(TV_Shows)'
conn.cursor().execute(sql2).fetchall()